# Semantic Router: Hybrid Layer

The Hybrid Layer in the Semantic Router library can improve decision making performance particularly for niche use-cases that contain specific terminology, such as finance or medical. It helps us provide more importance to decision making based on the keywords contained in our utterances and user queries.

## Getting Started

We start by installing the library:

In [ ]:
!pip install -qU semantic-router==0.0.5

We start by defining a dictionary mapping decisions to example phrases that should trigger those decisions.

In [1]:
import os

os.environ["COHERE_API_KEY"] = "<<APIKEY>>"

In [2]:
from semantic_router.schema import Decision

politics = Decision(
    name="politics",
    utterances=[
        "isn't politics the best thing ever",
        "why don't you tell me about your political opinions",
        "don't you just love the president" "don't you just hate the president",
        "they're going to destroy this country!",
        "they will save the country!",
    ],
)

/Users/jamesbriggs/opt/anaconda3/envs/decision-layer/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


Let's define another for good measure:

In [3]:
chitchat = Decision(
    name="chitchat",
    utterances=[
        "how's the weather today?",
        "how are things going?",
        "lovely weather today",
        "the weather is horrendous",
        "let's go to the chippy",
    ],
)

decisions = [politics, chitchat]

Now we initialize our embedding model:

In [4]:
from semantic_router.encoders import CohereEncoder
from getpass import getpass

os.environ["COHERE_API_KEY"] = os.environ["COHERE_API_KEY"] or getpass(
    "Enter Cohere API Key: "
)

encoder = CohereEncoder()

Now we define the `DecisionLayer`. When called, the decision layer will consume text (a query) and output the category (`Decision`) it belongs to — to initialize a `DecisionLayer` we need our `encoder` model and a list of `decisions`.

In [5]:
from semantic_router.layer import HybridDecisionLayer

dl = HybridDecisionLayer(encoder=encoder, decisions=decisions)

3454774732 not in encoder.idx_mapping


In [6]:
dl("don't you love politics?")

AxisError: axis 1 is out of bounds for array of dimension 1

---

#### Testing

In [ ]:
from semantic_router.encoders import BM25Encoder

encoder = BM25Encoder()

In [ ]:
tests = ["hello this is some text", "and more stuff"]

In [ ]:
idx_list = encoder.model.get_params()['doc_freq']['indices']
idx_list

In [ ]:
sparse_dicts = encoder.model.encode_documents(tests)
sparse_dicts

In [ ]:
embeds = [0.0] * len(encoder.idx_mapping)

In [ ]:
for output in sparse_dicts:
    indices = output["indices"]
    values = output["values"]
    for idx, val in zip(indices, values):
        position = encoder.idx_mapping[idx]
        embeds[position] = val

In [ ]:
encoder.idx_mapping

In [ ]:
encoded_output = encoder(tests)
encoded_output

In [ ]:
import numpy as np


sparse_vec = np.zeros(len(idx_list))
idx_position_dict = {idx: i for i, idx in enumerate(idx_list)}

for output in encoded_output:
    indices = output['indices']
    values = output['values']
    for idx, value in zip(indices, values):
        if idx in idx_position_dict:
            position = idx_position_dict[idx]
            sparse_vec[position] = value

In [ ]:
sparse_vec

In [ ]:
sparse_vec.shape

#### Finish Testing

---

Now we can test it:

In [ ]:
dl("don't you love politics?")

In [ ]:
dl("how's the weather today?")

Both are classified accurately, what if we send a query that is unrelated to our existing `Decision` objects?

In [ ]:
dl("I'm interested in learning about llama 2")

In this case, we return `None` because no matches were identified.